# <span style="color:#F72585"><center>Introducción a la API funcional de Keras</center></span>

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Redes_Neuronales/Imagenes/Funcional.JPG" width="700" height="400" align="center" /> 
</center>   
</figure>
<center>

Fuente: <a href="https://commons.wikimedia.org/wiki/File:Functional_group_with_sulfur_compounds_sulfone_sulfonic_acid_sulfonamide_2.JPG">Bin im Garten</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a>, via Wikimedia Commons

</center>

## <span style="color:#4361EE">Introducción</span>

Este es un notebook de Google Colaboratory. Los programas de Python se ejecutan directamente en tu navegador,  una gran manera de aprender y utilizar TensorFlow. Para poder seguir este tutorial, ejecuta este notebook en Google Colab. Basado en [Tensorflow- quick start expertos](https://www.tensorflow.org/tutorials/quickstart/advanced)


In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
print('Version de Tensorflow = ', tf.__version__)

# Objetos de la API de Keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

# datos mnist
from tensorflow.keras.datasets import mnist

# Manejo de datasets como tensores de tf
dataset = tf.data.Dataset.from_tensor_slices

# Métricas para medir pérdida y precisión
loss_metric     = tf.keras.metrics.Mean
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy

c:\Users\User\anaconda3\envs\work\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Version de Tensorflow =  2.9.1


## <span style="color:#4361EE">Prepara datos de MNIST</span> 

In [2]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# Agrega una dimensión para manejo de canales de imágenes en tensorflow
x_train = x_train[...,tf.newaxis]
x_test = x_test[...,tf.newaxis]

In [3]:
x_test.shape

(10000, 28, 28, 1)

## <span style="color:#4361EE">Separa lotes de datos  y mezclar el conjunto de datos</span>

Usamos *tf.data*.  Revise cuidadosamente el tutorial  [tf.data: compila canalizaciones de entrada de TensorFlow](https://www.tensorflow.org/guide/data)

In [4]:
train_ds = dataset((x_train, y_train)).shuffle(10000).batch(32)

test_ds  = dataset((x_test, y_test)).shuffle(10000).batch(32)

In [5]:
test_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

## <span style="color:#4361EE">API Funcional de Keras. Sub-clases</span>

Vamos a derivar nuestro model de la clase Model de la API funcional de Keras.

In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten= Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    
    def call(self,x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# crea una instancia del modelo
model = MyModel()

## <span style="color:#4361EE">Optimizador y función de pérdida</span>

Usaremos la entropía cruzada con varias clases (10 en este caso) y el optimizador Adam. La función *SparseCategoricalCrossentropy* recibe como entrada enteros y construye internamente la [codificación one-hot](https://en.wikipedia.org/wiki/One-hot). Puede usar la función *CategoricalCrossentropy* si desea integrar directamente las etiquetas en codificación one-hot.

Asegúrese de entender la *entropía cruzada* y su relación con la *codificación one-hot*. 

En caso de duda, consulte al instructor.

In [7]:
loss_object = SparseCategoricalCrossentropy()

optimizer = Adam()

Suponga por ejemplo que tiene tres categorías: 1,2,3. Discuta el siguiente fragmento (snippet) de código.
    

In [8]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

loss_object(y_true, y_pred).numpy()

1.1769392

Puede verificar que en este caso el resultado se obtiene de la siguiente forma. Explique por favor.

In [9]:
import numpy as np

-(np.log(0.95) + np.log(0.1))/2

1.176939193690798

O lo que es lo mismo

In [10]:
-(np.log(y_pred[0][y_true[0]]) + np.log(y_pred[1][y_true[1]]))/len(y_pred)

1.176939193690798

## <span style="color:#4361EE">Métricas para medir pérdida y precisión</span>

Escoge métricas para medir la perdida y exactitud del modelo. Estas métricas acumulan los valores cada epoch y después imprimen el resultado total.


In [11]:
train_loss = loss_metric(name='train_loss')
train_accuracy = accuracy_metric(name='train_accuracy')

test_loss = loss_metric(name='test_loss')
test_accuracy = accuracy_metric(name='loss_accuracy')


## <span style="color:#4361EE">Función de entrenamiento: diferenciación automática con tf.GradientTape</span>

La función de entrenamiento es decorada con el decorador *@tf.function*  Este decorado compila la función como un grafo de TensorFlow invocable.

In [12]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
    

## <span style="color:#4361EE">Función de prueba (test)</span>

In [13]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

## <span style="color:#4361EE">Entrena el modelo</span>

In [14]:
def fit(train_dataset, test_dataset, epochs):
    for epoch in range(epochs):
        for images, labels in train_dataset:
            train_step(images, labels)
        
        for images, labels in test_dataset:
            test_step(images, labels)
        
        template = 'Epoch {}, Pérdida: {}, Exactitud: {}, Pérdida de prueba: {}, Exactitud de prueba {}'
        print(template.format(epoch+1,
                              train_loss.result(),
                              train_accuracy.result(),
                              test_loss.result(),
                              test_accuracy.result()))
        
        # Reinicia las métricas para el siguiente paso
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [15]:
fit(train_ds, test_ds, epochs = 5)

Epoch 1, Pérdida: 0.13836877048015594, Exactitud: 0.9588500261306763, Pérdida de prueba: 0.06696019321680069, Exactitud de prueba 0.9782000184059143
Epoch 2, Pérdida: 0.042073335498571396, Exactitud: 0.9869499802589417, Pérdida de prueba: 0.049123987555503845, Exactitud de prueba 0.9832000136375427
Epoch 3, Pérdida: 0.020745323970913887, Exactitud: 0.9932833313941956, Pérdida de prueba: 0.05571064352989197, Exactitud de prueba 0.9828000068664551
Epoch 4, Pérdida: 0.012428789399564266, Exactitud: 0.9957666397094727, Pérdida de prueba: 0.062411025166511536, Exactitud de prueba 0.9830999970436096
Epoch 5, Pérdida: 0.008782007731497288, Exactitud: 0.9970999956130981, Pérdida de prueba: 0.07019472122192383, Exactitud de prueba 0.9803000092506409
